In [11]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.emb_auto import AutoDataset
from Model.AE import Auto

import utils

import matplotlib.pyplot as plt

In [12]:
# 데이터 다운 & 전처리

df = pd.read_excel('../데이터/더미데이터.xlsx')

try: del df['Unnamed: 0']
except: pass

In [13]:
df[0] = df[0].apply(utils.to_list)
df[1] = df[1].apply(utils.to_list)

In [14]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 64

train_dataset = AutoDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = AutoDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = AutoDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [15]:
for x,y in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  break

X 크기 : torch.Size([12, 7])
Y 크기 : torch.Size([12, 7])


In [16]:
input_size = 7
Model = Auto(input_size)

In [17]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,input_size))
model(basic_data)


cpu is available
작동하는지 실험


(tensor([[0.3239, 0.0000]], grad_fn=<ReluBackward0>),
 tensor([[0.4480, 0.4892, 0.6975, 0.5566, 0.4091, 0.6063, 0.3425]],
        grad_fn=<SigmoidBackward0>))

In [18]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [19]:
## 학습하는 코드를 짜봐!!!

train_n = len(train_loader)
val_n = len(val_loader)

# print(train_loader)
best_val_loss = np.inf

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        inputs = inputs.to(device)
        labels = labels.to(device)
        vec, out = model(inputs)
        loss = criterion(out, labels)

        optimizer.zero_grad() #
        loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss.item() # 한 배치의 loss 더해주고,


    print(running_loss/train_n)

    with torch.no_grad():
              print("Calculating validation results...")
              model.eval()
              val_loss = 0
              for inputs, labels in val_loader:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  vec, outs = model(inputs)

                  loss= criterion(outs, labels)
                  val_loss += loss.item()



              # if val_acc > best_val_acc:
              #     print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
              #     # torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
              #     best_val_acc = val_acc
              # torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
              print("val")
              print(
                  f"loss: {val_loss/val_n:4.8} || "
              )
    print('======================================================')



epoch : 0
train
670541568.0
Calculating validation results...
val
loss: 7.8448179e+08 || 


In [20]:
for x,y in test_dataset:
    print(model(x))
    break

(tensor([   0.0000, 3218.5120], grad_fn=<ReluBackward0>), tensor([3.0467e-14, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
        1.0000e+00], grad_fn=<SigmoidBackward0>))
